In [1]:
from transformers import AutoModelForVision2Seq, AutoProcessor, AwqConfig
from PIL import Image
import torch
import os 

os.environ["TRANSFORMERS_OFFLINE"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from awq import AutoAWQForCausalLM

/home/zhekai/miniforge3/envs/openvla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-04 23:05:06.274241: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-04 23:05:06.308510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 23:05:06.308543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 23:

AWQ quantization (4-bits), not using flashattention

In [5]:
# Load Processor & VLA
quantization_config = AwqConfig(
    bits=4,
    fuse_max_seq_len=16,
    do_fuse=True,
)

quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version":"GEMM"}

# quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=10.0)
model_path = "openvla/openvla-7b"
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
    low_cpu_mem_usage=True, 
    trust_remote_code=True,
    # quantization_config=quantization_config,
)#.to(device)

# Grab image input & format prompt
image = Image.open("test.png")
instruction = "put eggplant into pot"
prompt = f"In: What action should the robot take to {instruction}?\nOut:"



Expected `transformers==4.40.1` and `tokenizers==0.19.1` but got `transformers==4.46.1` and `tokenizers==0.20.1`; there might be inference-time regressions due to dependency changes. If in doubt, pleaseuse the above versions.
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.89it/s]


In [ ]:
# from torch.profiler import profile, record_function, ProfilerActivity

# === BFLOAT16 MODE ===
inputs = processor(prompt, image).to(device, dtype=torch.bfloat16)
# inputs["input_ids"] = inputs["input_ids"][:, 1:]

# Run OpenVLA Inference
torch.manual_seed(0)

def trace_handler(prof):
    # print(prof.key_averages().table(
    #     sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace("tmp/test_trace_" + str(prof.step_num) + ".json")

# with torch.profiler.profile(
#     activities=[
#         torch.profiler.ProfilerActivity.CPU,
#         torch.profiler.ProfilerActivity.CUDA,
#     ],
#     schedule=torch.profiler.schedule(
#         wait=1,
#         warmup=1,
#         active=1),
#     on_trace_ready=trace_handler,
#     with_stack=True,
#     profile_memory=True,
#     with_flops = True
#     ) as p:
#         for iter in range(3):
#             action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)
#             p.step()



In [ ]:
# profile latecy with cuda event
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)
end.record()
torch.cuda.synchronize()
print(start.elapsed_time(end))

200.7198028564453


In [ ]:
#profile memroy with torch
print(torch.cuda.memory_summary(device))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   4478 MiB |   4762 MiB |  56037 MiB |  51559 MiB |
|       from large pool |   4271 MiB |   4555 MiB |  54753 MiB |  50481 MiB |
|       from small pool |    206 MiB |    210 MiB |   1284 MiB |   1077 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   4478 MiB |   4762 MiB |  56037 MiB |  51559 MiB |
|       from large pool |   4271 MiB |   4555 MiB |  54753 MiB |